In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, chi2_contingency, f_oneway, mannwhitneyu, kruskal

In [17]:
# Load cleaned dataset
df = pd.read_csv(r"C:\Users\hp\Desktop\10 Acadamy\VS code\insurance-risk-analytics\insurance-risk-analytics-week3\data\insurance_data_eda.csv")

C:\Users\hp\AppData\Local\Temp\ipykernel_16784\2291897501.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\hp\Desktop\10 Acadamy\VS code\insurance-risk-analytics\insurance-risk-analytics-week3\data\insurance_data_eda.csv")


In [18]:
# Preprocessing: Create necessary metrics
df['ClaimOccurred'] = (df['TotalClaims'] > 0).astype(int)
df['ClaimSeverity'] = df['TotalClaims'] / df['ClaimOccurred'].replace(0, np.nan)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

In [19]:
# Claim Frequency (Chi-squared test)
province_claims = pd.crosstab(df['Province'], df['ClaimOccurred'])
chi2_stat, chi2_p, _, _ = chi2_contingency(province_claims)

In [20]:
# Claim Severity (ANOVA)
severity_groups = [group['ClaimSeverity'].dropna() for name, group in df.groupby('Province')]
anova_stat, anova_p = f_oneway(*severity_groups)

In [21]:
print("H₀₁: Province vs Risk")
print(f"Chi-squared p-value (Claim Frequency): {chi2_p:.4f}")
print(f"ANOVA p-value (Claim Severity): {anova_p:.4f}")
print("Reject H₀" if chi2_p < 0.05 or anova_p < 0.05 else "Fail to reject H₀")
print()

H₀₁: Province vs Risk
Chi-squared p-value (Claim Frequency): 0.0000
ANOVA p-value (Claim Severity): 0.0000
Reject H₀



H₀₂: No Risk Differences Between Zipcodes

In [22]:
top_zips = df['PostalCode'].value_counts().head(10).index
df_zip = df[df['PostalCode'].isin(top_zips)]

In [23]:
# Claim Frequency (Chi-squared test)
zip_claims = pd.crosstab(df_zip['PostalCode'], df_zip['ClaimOccurred'])
chi2_zip_stat, chi2_zip_p, _, _ = chi2_contingency(zip_claims)

In [24]:
# Claim Severity (ANOVA)
zip_groups = [group['ClaimSeverity'].dropna() for name, group in df_zip.groupby('PostalCode')]
anova_zip_stat, anova_zip_p = f_oneway(*zip_groups)




In [25]:
print("H₀₂: Zipcode vs Risk")
print(f"Chi-squared p-value (Claim Frequency): {chi2_zip_p:.4f}")
print(f"ANOVA p-value (Claim Severity): {anova_zip_p:.4f}")
print("Reject H₀" if chi2_zip_p < 0.05 or anova_zip_p < 0.05 else "Fail to reject H₀")
print()

H₀₂: Zipcode vs Risk
Chi-squared p-value (Claim Frequency): 0.0000
ANOVA p-value (Claim Severity): 0.0000
Reject H₀



H₀₃: No Significant Margin Difference Between Zipcodes

In [26]:
margin_groups = [group['Margin'].dropna() for name, group in df_zip.groupby('PostalCode')]
kruskal_stat, kruskal_p = kruskal(*margin_groups)

print("H₀₃: Zipcode vs Margin")
print(f"Kruskal-Wallis p-value: {kruskal_p:.4f}")
print("Reject H₀" if kruskal_p < 0.05 else "Fail to reject H₀")
print()

H₀₃: Zipcode vs Margin
Kruskal-Wallis p-value: 0.0000
Reject H₀



H₀₄: No Significant Risk Difference Between Women and Men

In [27]:
# Filter gender groups
female = df[df['Gender'] == 'Female']
male = df[df['Gender'] == 'Male']

In [28]:
# Claim Frequency (Chi-squared)
gender_claims = pd.crosstab(df['Gender'], df['ClaimOccurred'])
chi2_gender_stat, chi2_gender_p, _, _ = chi2_contingency(gender_claims)

In [29]:
# Claim Severity (T-test)
t_stat_gender, t_p_gender = ttest_ind(
    female['ClaimSeverity'].dropna(),
    male['ClaimSeverity'].dropna(),
    equal_var=False
)

print("H₀₄: Gender vs Risk")
print(f"Chi-squared p-value (Claim Frequency): {chi2_gender_p:.4f}")
print(f"T-test p-value (Claim Severity): {t_p_gender:.4f}")
print("Reject H₀" if chi2_gender_p < 0.05 or t_p_gender < 0.05 else "Fail to reject H₀")
print()

H₀₄: Gender vs Risk
Chi-squared p-value (Claim Frequency): 0.0266
T-test p-value (Claim Severity): 0.5680
Reject H₀



In [30]:
# Save results to CSV or Markdown table
results = [
    ['H₀₁', chi2_p, anova_p, chi2_p < 0.05 or anova_p < 0.05],
    ['H₀₂', chi2_zip_p, anova_zip_p, chi2_zip_p < 0.05 or anova_zip_p < 0.05],
    ['H₀₃', None, kruskal_p, kruskal_p < 0.05],
    ['H₀₄', chi2_gender_p, t_p_gender, chi2_gender_p < 0.05 or t_p_gender < 0.05],
]

result_df = pd.DataFrame(results, columns=['Hypothesis', 'p_val_1', 'p_val_2', 'Reject_H₀'])
result_df.to_csv("task3_hypothesis_results.csv", index=False)